In [ ]:


import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math

import torch
import torch.nn as nn
import torch.utils 
import torch.utils.data
from torch.autograd import Variable
import os
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


In [ ]:
# Some lines copied from:
# https://www.kaggle.com/pavfedotov/lstm-to-predict-bitcoin-price

In [ ]:
data = pd.read_csv("/kaggle/input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")
data

In [ ]:
def plotfig(data):
    plt.figure(figsize = (16,8))
    plt.plot(data["Weighted_Price"])
    plt.xlabel('Time',fontsize=15)
    plt.ylabel('Weighted_Price',fontsize=15)
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
data["Timestamp"] = pd.to_datetime(data["Timestamp"], infer_datetime_format=True, unit="s")
data = data.set_index("Timestamp")
plotfig(data)
data

In [ ]:
# using only 100 days of data

data = data.tail(100*24*60+1)
plotfig(data)

In [ ]:
#taking interval of 10 minutes

data = data[::10]
data.dropna(axis=0,inplace=True)


In [ ]:
data

In [ ]:
# data['date'] = pd.to_datetime(data['Timestamp'],unit='s').dt.date
# # changing per minute data to per day
# group = data.groupby('date')

# data["Timestamp"] = pd.to_datetime(data["Timestamp"], infer_datetime_format=True, unit="s")
# data = data.set_index("Timestamp")

# data["Timestamp"] = pd.to_datetime(data["Timestamp"], infer_datetime_format=True, unit="s")
# data = data.set_index("Timestamp")
# # data.groupby(data.index.hour).mean()
# # data.groupby(pd.Grouper(key='Timestamp',freq='H')).mean()
# # data

# times = pd.to_datetime(data.datetime_col)
# data.groupby([times.hour, times.minute]).value_col.mean()

# Real_Price = group['Weighted_Price'].mean()
# Real_Price

In [ ]:
#since we need only Weighted_Price, we drop all other

data = data["Weighted_Price"]
data

In [ ]:
# predicting last 5 days
pred_days = 5
test_size = int(pred_days*24*60/10)

X_train = data[:len(data)-test_size].values.reshape(-1,1)
X_test = data[len(data)-test_size:].values.reshape(-1,1)
print(X_train.shape)
print(X_test.shape)


In [ ]:
X_train

In [ ]:
#scaling

# scaler_train = MinMaxScaler(feature_range=(0, 1))
# X_train_scaled = scaler_train.fit_transform(X_train)

# scaler_test = MinMaxScaler(feature_range=(0, 1))
# X_test_scaled = scaler_test.fit_transform(X_test)

In [ ]:
# getting trainX and trainY
# trainY is next element of corresponding trainX

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

trainX, trainY = create_dataset(X_train_scaled)
testX, testY = create_dataset(X_test_scaled)

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
trainX.shape

# Keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM


regressor = Sequential()
regressor.add(LSTM(units = 128, activation = 'relu',return_sequences=True, input_shape = (trainX.shape[1], trainX.shape[2])))
regressor.add(LSTM(units = 64, input_shape = (trainX.shape[1], trainX.shape[2])))
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
history = regressor.fit(trainX, trainY, batch_size = 32, epochs = 30, verbose=1, shuffle=False, validation_data=(testX, testY))

In [ ]:
from matplotlib import pyplot

import plotly.offline as py
import plotly.graph_objs as go
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# Making the predictions
predicted_BTC_price = regressor.predict(testX)
predicted_BTC_price = scaler_test.inverse_transform(predicted_BTC_price.reshape(-1, 1))

true = scaler_test.inverse_transform(testY.reshape(-1, 1))

In [ ]:
pyplot.plot(predicted_BTC_price, label='predict')
pyplot.plot(true, label='true')
pyplot.legend()
pyplot.show()

In [ ]:
from math import sqrt
rmse = sqrt(mean_squared_error(true, predicted_BTC_price))
print('Test RMSE: %.3f' % rmse)